In [135]:
import os
from dotenv import load_dotenv

In [136]:

load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")


In [137]:
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [138]:
from langchain_community.document_loaders import PyPDFLoader


In [139]:
%cd D:/PG-AGI-Interview-Assistant

D:\PG-AGI-Interview-Assistant


In [140]:
%pwd

'D:\\PG-AGI-Interview-Assistant'

In [141]:
file_path = "data/SDG.pdf"
loader = PyPDFLoader(file_path)

In [142]:
data = loader.load()

In [143]:
data

[Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-12-24T16:02:52+00:00', 'source': 'data/SDG.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content='4th SDG Y outh Summer Camp – SDG Resource Document The 2030 Agenda for Sustainable Development’s 17 Sustainable Development Goals (SDGs)   Goal: This document enables 4th SDG Youth Summer Camp participants to i) get to know the 17 SDGs, ii) explore what areas each goal covers under its targets, iii) identify targets of most interest to participants, and iv) identify synergies between the SDGs and chosen target(s).    Goal 1. End poverty in all its forms everywhere  Target 1.1 By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day  Target 1.2 By 2030, reduce at least by half the proportion of men, women and children of all ages living in poverty in all its dimensions according to national definitions  Target 1.

In [144]:
question_gen = ""
for page in data :
    question_gen += page.page_content + "\n\n"

In [145]:
question_gen

'4th SDG Y outh Summer Camp – SDG Resource Document The 2030 Agenda for Sustainable Development’s 17 Sustainable Development Goals (SDGs)   Goal: This document enables 4th SDG Youth Summer Camp participants to i) get to know the 17 SDGs, ii) explore what areas each goal covers under its targets, iii) identify targets of most interest to participants, and iv) identify synergies between the SDGs and chosen target(s).    Goal 1. End poverty in all its forms everywhere  Target 1.1 By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day  Target 1.2 By 2030, reduce at least by half the proportion of men, women and children of all ages living in poverty in all its dimensions according to national definitions  Target 1.3 Implement nationally appropriate social protection systems and measures for all, including floors, and by 2030 achieve substantial coverage of the poor and the vulnerable  Target 1.4 By 2030, ensure that all me

In [146]:
from langchain_text_splitters import TokenTextSplitter

In [147]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Approximate tokens (Llama uses ~4 chars per token)
splitter_ques_gen = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # ~10k tokens worth of characters
    chunk_overlap=150,
)

In [148]:
chunks_ques_gen = splitter_ques_gen.split_text(question_gen)

In [149]:
len(chunks_ques_gen)

16

In [150]:
from langchain_community.docstore.document import Document


In [151]:
document_ques_gen = [Document(page_content=chunk) for chunk in chunks_ques_gen]

In [152]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Approximate tokens (Llama uses ~4 chars per token)
splitter_ans_gen = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # ~10k tokens worth of characters
    chunk_overlap=100,
)

In [153]:
document_ans_gen = splitter_ans_gen.split_documents(document_ques_gen)

In [154]:
len(document_ans_gen)

16

In [155]:
from langchain_groq import ChatGroq

In [156]:
llm_ques_gen_pipeline = ChatGroq(
    model="llama-3.3-70b-versatile",
    groq_api_key=GROQ_API_KEY,  # or set GROQ_API_KEY env variable
    temperature=0.3
)

In [157]:
prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""


In [158]:
from langchain_core.prompts import PromptTemplate

In [159]:
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=["text"])


In [160]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones,
(only if necessary) with some more context below.

{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.

QUESTIONS:
""")


In [161]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)


In [162]:
from langchain_classic.chains.summarize import load_summarize_chain

ques_gen_chain = load_summarize_chain(
    llm=llm_ques_gen_pipeline,
    chain_type="refine",
    verbose=True,
    question_prompt=PROMPT_QUESTIONS,
    refine_prompt=REFINE_PROMPT_QUESTIONS
)

In [ ]:
ques = ques_gen_chain.run(document_ques_gen)

print(ques)




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
4th SDG Y outh Summer Camp – SDG Resource Document The 2030 Agenda for Sustainable Development’s 17 Sustainable Development Goals (SDGs)   Goal: This document enables 4th SDG Youth Summer Camp participants to i) get to know the 17 SDGs, ii) explore what areas each goal covers under its targets, iii) identify targets of most interest to participants, and iv) identify synergies between the SDGs and chosen target(s).    Goal 1. End poverty in all its forms everywhere  Target 1.1 By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day  Target 1.2 By 2030, reduce at least by half the proport

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jm7cpz9qfbfsn3mpemj24j2x` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 100000, Requested 32965. Please try again in 7h54m41.76s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [165]:
from langchain_huggingface import HuggingFaceEmbeddings

In [166]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

d:\PG-AGI-Interview-Assistant\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [168]:
from langchain_community.vectorstores import FAISS

In [169]:
vector_store = FAISS.from_documents(document_ans_gen, embeddings)

In [170]:
llm_answer_gen = ChatGroq(
    temperature=0.1,
    model="llama-3.3-70b-versatile",  # or "llama-3.1-8b-instant"
    groq_api_key="GROQ_API_KEY",  # or set GROQ_API_KEY env variable
)

In [ ]:
ques

In [ ]:
ques_list = ques.split("\n")

In [171]:
from langchain_classic.chains import RetrievalQA

In [172]:
answer_generation_chain = RetrievalQA.from_chain_type(
    llm=llm_answer_gen,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)


In [ ]:
for question in ques_list:
    print("Question:", question)
    answer = answer_generation_chain.run(question)
    print("Answer:", answer)
    print("------------------------------")

    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\n")
        f.write("Answer: " + answer + "\n")
        f.write("------------------------------\n")
